# Mushrooms

Brendan Reed  
11/11/2025  

Attempts to correctly identify mushrooms as poisonous or non-poisonous using classification.

In [ ]:
# Imports


## Section 1. Import and Inspect the Data

### 1.1 Load the dataset and display the first 10 rows.

### 1.2 Check for missing values and display summary statistics.

Reflection 1: What do you notice about the dataset? Are there any data issues?

## Section 2. Data Exploration and Preparation

### 2.1 Explore data patterns and distributions

### 2.2 Handle missing values and clean data


### 2.3 Feature selection and engineering

Reflection 2: What patterns or anomalies do you see? Do any features stand out? What preprocessing steps were necessary to clean and improve the data? Did you create or modify any features to improve performance?

## Section 3. Feature Selection and Justification

### 3.1 Choose features and target

### 3.2 Define X and y


Reflection 3: Why did you choose these features? How might they impact predictions or accuracy?

### 4.1 Split the data

### 4.2 Train model

### 4.3 Evalulate performance

## Section 5. Improve the Model or Try Alternates

5.1 Train an alternative classifier (e.g., Decision Tree, Random Forest, Logistic Regression) OR adjust hyperparameters on the original model.

5.2 Compare performance of all models across the same performance metrics.

Reflection 5: Which model performed better? Why might one classifier be more effective in this specific case?



## Section 6. Final Thoughts & Insights

### 6.1 Summarize findings

### 6.2 Discuss challenges faced

### 6.3 If you had more time, what would you try next?

Reflection 6: What did you learn from this project?